# Follow-Me Project
Congratulations on reaching the final project of the Robotics Nanodegree! 

Previously you worked on the Semantic Segmentation lab where you built a deep learning network that locates a particular human target within an image. For this project, you will utilize what you implemented and learned from that lab and extend it to train a deep learning model that will allow a simulated quadcopter to follow around the person that it detects! 

Most of the code below will be familiar in comparison to the lab with some minor modifications. So you can work off of your existing solution, and modify and improve upon it to train the best possible model for this task.

## Data Collection
We have provided you with a dataset for this project. If you haven't already downloaded the training and validation datasets, you can check out the README for this project's repo for instructions as well.

Alternatively, you can also collect your own data and see how well your model can perform on that or tweak your model accordingly. You can check out the "Collecting Data" section in the Project Lesson in the Classroom for more details!

In [ ]:
import os
import glob
import sys
import tensorflow as tf

from scipy import misc
import numpy as np

from tensorflow.contrib.keras.python import keras
from tensorflow.contrib.keras.python.keras import layers, models

from tensorflow import image

from utils import scoring_utils
from utils.separable_conv2d import SeparableConv2DKeras, BilinearUpSampling2D
from utils import data_iterator

## FCN Layers
In the Classroom, we discussed the different layers that constitute a fully convolutional network. The following code will intoduce you to the functions that you will be using to build out your model.

### Separable Convolutions
The Encoder for your FCN will essentially require separable convolution layers. Below we have implemented two functions - one which you can call upon to build out separable convolutions or regular convolutions. Each with batch normalization and with the ReLU activation function applied to the layers. 

While we recommend the use of separable convolutions thanks to their advantages we covered in the Classroom, some of the helper code we will present for your model will require the use for regular convolutions. But we encourage you to try and experiment with each as well!

The following will help you create the encoder block and the final model for your architecture.

In [ ]:
def separable_conv2d_batchnorm(input_layer, filters, strides=1):
    output_layer = SeparableConv2DKeras(filters=filters,kernel_size=3, strides=strides,
                             padding='same', activation='relu')(input_layer)
    
    output_layer = layers.BatchNormalization()(output_layer) 
    return output_layer

def conv2d_batchnorm(input_layer, filters, kernel_size=3, strides=1):
    output_layer = layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=1, 
                      padding='same', activation='relu')(input_layer)
    
    output_layer = layers.BatchNormalization()(output_layer) 
    return output_layer

### Bilinear Upsampling
The following helper function will help implement the bilinear upsampling layer. Currently, upsampling by a factor of 2 is recommended but you can try out different factors as well. You will use this to create the decoder block later!

In [ ]:
def bilinear_upsample(input_layer):
    output_layer = BilinearUpSampling2D((2,2))(input_layer)
    return output_layer

## Build the Model
In the following cells, we will cover how to build the model for the task at hand. 

- We will first create an Encoder Block, where you will create a separable convolution layer using an input layer and the size(depth) of the filters as your inputs.
- Next, you will create the Decoder Block, where you will create an upsampling layer using bilinear upsampling, followed by a layer concatentaion, and some separable convolution layers.
- Finally, you will combine the above two and create the model. In this step you will be able to experiment with different number of layers and filter sizes for each to build your model.

Let's cover them individually below.

### Encoder Block
Below you will create a separable convolution layer using the separable_conv2d_batchnorm() function. The `filters` parameter defines the size or depth of the output layer. For example, 32 or 64. 

In [ ]:
def encoder_block(input_layer, filters, strides):
    
    # TODO Create a separable convolution layer using the separable_conv2d_batchnorm() function.
    
    return output_layer

### Decoder Block
The decoder block, as covered in the Classroom, comprises of three steps -

- A bilinear upsampling layer using the upsample_bilinear() function. The current recommended factor for upsampling is set to 2.
- A layer concatenation step. This step is similar to skip connections. You will concatenate the upsampled small_ip_layer and the large_ip_layer.
- Some (one or two) additional separable convolution layers to extract some more spatial information from prior layers.

In [ ]:
def decoder_block(small_ip_layer, large_ip_layer, filters):
    
    # TODO Upsample the small input layer using the bilinear_upsample() function.
    
    # TODO Concatenate the upsampled and large input layers using layers.concatenate
    
    # TODO Add some number of separable convolution layers
    
    return output_layer

### Model

Now that you have the encoder and decoder blocks ready, you can go ahead and build your model architecture! 

There are three steps to the following:
- Add encoder blocks to build out initial set of layers. This is similar to how you added regular convolutional layers in your CNN lab.
- Add 1x1 Convolution layer using conv2d_batchnorm() function. Remember that 1x1 Convolutions require a kernel and stride of 1.
- Add decoder blocks for upsampling and skip connections.

In [ ]:
def fcn_model(inputs, num_classes):
    
    # TODO Add Encoder Blocks. 
    # Remember that with each encoder layer, the depth of your model (the number of filters) increases.

    # TODO Add 1x1 Convolution layer using conv2d_batchnorm().
    
    # TODO: Add the same number of Decoder Blocks as the number of Encoder Blocks
    
    
    # The function returns the output layer of your model. "x" is the final layer obtained from the last decoder_block()
    return layers.Conv2D(num_classes, 1, activation='softmax', padding='same')(x)

## Training
The following cells will utilize the model you created and define an ouput layer based on the input and the number of classes.Following that you will define the hyperparameters to compile and train your model!

Please Note: For the project you will be working with images of dimensions 160x160x3.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""

image_hw = 160
image_shape = (image_hw, image_hw, 3)
inputs = layers.Input(image_shape)
num_classes = 3

# Call fcn_model()
output_layer = fcn_model(inputs, num_classes)

### Hyperparameters
Below you can define and tune your hyperparameters

In [ ]:
learning_rate = 0
batch_size = 0
num_epochs = 0

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# Define the Keras model and compile it for training
model = models.Model(inputs=inputs, outputs=output_layer)

model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='categorical_crossentropy')

# Data iterators for loading the training and validation data
train_iter = data_iterator.BatchIteratorSimple(batch_size=batch_size,
                                               data_folder=os.path.join('..', 'data', 'train'),
                                               image_shape=image_shape,
                                               shift_aug=True)

val_iter = data_iterator.BatchIteratorSimple(batch_size=batch_size,
                                             data_folder=os.path.join('..', 'data', 'validation'),
                                             image_shape=image_shape)

model.fit_generator(train_iter,
                    steps_per_epoch = 350, # the number of batches per epoch,
                    epochs = num_epochs, # the number of epochs to train for,
                    validation_data = val_iter, # validation iterator
                    validation_steps = 75, # the number of batches to validate on
                    workers = 5)

In [ ]:
# Save your trained model weights
weight_file_name = 'model_weights'
model.save_weights(os.path.join('..', 'data', 'weights', weight_file_name))

## Prediction

In [ ]:
# If you need to load a model which you previously trained you can uncomment the codeline that calls the following function.
def load_weights(your_model, your_weight_filename):
    model_path = os.path.join('..', 'data', 'weights', your_weight_filename)
    if os.path.exists(model_path):
        model = your_model.load_weights(model_path)
        return model
    else:
        raise ValueError('No weight file found at {}'.format(model_path))

# model = load_weights(model, weight_file_name)

In [ ]:
def make_dir_if_not_exist(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def predict(filenames, output_path):
    for name in file_names:
        image = misc.imread(name)
        if image.shape[0] != image_shape[0]:
             image = misc.imresize(image, image_shape)
        image = data_iterator.preprocess_input(image.astype(np.float32))
        pred = model.predict_on_batch(np.expand_dims(image, 0))
        base_name = os.path.basename(name).split('.')[0]
        base_name = base_name + '_prediction.png'
        misc.imsave(os.path.join(output_path, base_name), np.squeeze((pred * 255).astype(np.uint8)))

validation_path1 = os.path.join('..', 'data', 'grading_data1', 'patrol_with_targ')
file_names = sorted(glob.glob(os.path.join(validation_path1, 'images', '*.jpeg')))

experiment_name = 'patrol_with_targ'# TODO add the name of folder to save these predictions to
output_path1 = os.path.join('..', 'data', 'runs', experiment_name)
make_dir_if_not_exist(output_path1)
predict(file_names, output_path1)

validation_path2 = os.path.join('..', 'data', 'grading_data1', 'patrol_non_targ')
file_names = sorted(glob.glob(os.path.join(validation_path2, 'images', '*.jpeg')))

experiment_name = 'patrol_no_targ'# TODO add the name of folder to save these predictions to
output_path2 = os.path.join('..', 'data', 'runs', experiment_name)
make_dir_if_not_exist(output_path2)
predict(file_names, output_path2)

validation_path3 = os.path.join('..', 'data', 'grading_data1', 'following_images')
file_names = sorted(glob.glob(os.path.join(validation_path3, 'images', '*.jpeg')))

experiment_name = 'following_image'# TODO add the name of folder to save these predictions to
output_path3 = os.path.join('..', 'data', 'runs', experiment_name)
make_dir_if_not_exist(output_path3)
predict(file_names, output_path3)

## Evaluation
Let's evaluate your model!

In [ ]:
# scores for while the quad is following behind the target. 
scoring_utils.score_run_iou(validation_path3, output_path3)
print('\n')
# the centroid accuracy is important to look at here. The small the centroid error the more 
# accurately the targets x,y,z coordinates can be calculated
scoring_utils.score_run_centroid(validation_path3, output_path3)

# scores for images while the quad is on patrol and the target is not visable
scoring_utils.score_run_iou(validation_path2, output_path2)
print('\n')

# the most important score here is the number of false positives which indicates
# How often the network would tell the quad to go towards the wrong person
scoring_utils.score_run_centroid(validation_path3, output_path3)

# this score measures how well the neural network can detect the target from far away
scoring_utils.score_run_iou(validation_path1, output_path1)
print('\n')

# Here there is an emphasis on looking at the false negatives, how often does the network, miss 
# the target when they are are actually in view
scoring_utils.score_run_centroid(validation_path1, output_path1)

# additionally the centroid errors are an indicator of the accuracy of x,y,z of the targets coordinates